In [1]:
# !pip install xgboost
#!pip install scikit-learn
# !pip install nltk
# !pip install shap

In this notebook, we calculate the Population Stability Index (PSI) to assess the stability of population distributions between the training and test datasets.

### Libraries
We import necessary libraries for data manipulation and analysis.

### Bin Calculation
Each feature is discretized into bins to standardize the distribution of continuous variables.

### Probability Calculation
We compute the probability distribution for each bin in the training dataset, serving as the reference for comparison.

### PSI Calculation
Using the reference probabilities, we calculate the Population Stability Index (PSI) for each feature between the training and test datasets.

This analysis enables us to identify any significant shifts or skews in the population distributions, ensuring the reliability of our analytical models.

In [2]:
from google.cloud import bigquery
from google.oauth2 import service_account
import os
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [3]:
from config import credential_datamart, project_id_bq, churn_bq_db, churn_result_table, churn_score_result_table
from module.utils import memory_usage, read_file, binary_label_summary
# from module.viz_utils import metric_for_train_df, metric_for_test_df, conf_matrix_viz, shap_viz, correlation_plot, feature_importance_plot

In [4]:
credentials = service_account.Credentials.from_service_account_file(
    credential_datamart, scopes = ["https://www.googleapis.com/auth/cloud-platform"],
)
bq_client = bigquery.Client(project = project_id_bq, credentials = credentials)

In [5]:
# fetch data
query = """
SELECT  *
FROM
 `{project_id_bq}.{churn_bq_db}.{churn_result_table}`
WHERE type="TRAIN"
""".format(project_id_bq = project_id_bq, churn_bq_db = churn_bq_db, churn_result_table = churn_result_table)

train_df = bq_client.query(query).to_dataframe()



In [6]:
read_file(train_df)

SHAPE (2576, 16)
no missing value
TYPE
SUM_PROMO_last90               float64
SUM_GMV_last90                 float64
SUM_PROMO_last30days           float64
n_month_active                   Int64
SUM_PRDTRX_last30days            Int64
SUM_PRDTRX_PROMO_last30days      Int64
SUM_GMV_last30days             float64
SUM_GMV_growth                 float64
PROMO_ratio_last30days         float64
SUM_PROMO_growth               float64
type                            object
customer_id                     object
week_date                       dbdate
date                            dbdate
is_churned                       Int64
proba                          float64
dtype: object


In [7]:
memory_usage()

,Size
__builtins__,9.09KB
datetime,416.00B
__file__,219.00B
timer,136.00B
binary_label_summary,136.00B
memory_usage,136.00B
obj_size_fmt,136.00B
read_file,136.00B
__cached__,129.00B
gc,72.00B


In [8]:
# list of features used 
col_to_be_trained = ['SUM_PROMO_last90',
 'SUM_GMV_last90',
 'SUM_PROMO_last30days',
 'n_month_active',
 'SUM_PRDTRX_last30days',
 'SUM_PRDTRX_PROMO_last30days',
 'SUM_GMV_last30days',
 'SUM_GMV_growth',
 'PROMO_ratio_last30days',
 'SUM_PROMO_growth',
                    ]

In [9]:
numer_feature = ['SUM_PROMO_last90',
 'SUM_GMV_last90',
 'SUM_PROMO_last30days',
 # 'n_month_active',
 'SUM_PRDTRX_last30days',
 'SUM_PRDTRX_PROMO_last30days',
 'SUM_GMV_last30days',
 'SUM_GMV_growth',
 'PROMO_ratio_last30days',
 'SUM_PROMO_growth',
 'proba'
                    ]

In [10]:
min_max_df=pd.DataFrame({'min':train_df[numer_feature].min(),'max':train_df[numer_feature].max()})

In [11]:
min_max_df['step']=round((min_max_df['max']-min_max_df['min'])/9,3)

In [12]:
feature_min_max_dict={}
for i in numer_feature:
    feature_min_max_dict[i]=np.arange(min_max_df.loc[i,'min'],min_max_df.loc[i,'max']+min_max_df.loc[i,'step'],min_max_df.loc[i,'step'])

In [13]:
for i in col_to_be_trained:
    if i not in numer_feature:
        feature_min_max_dict[i]=[0,1,2,3]

In [14]:
for i in feature_min_max_dict.keys():
    if i in numer_feature:
        if len(feature_min_max_dict[i])==10:
            min_max_df.loc[i,'step']=round((min_max_df.loc[i,'max']-min_max_df.loc[i,'min'])/10,3)
            feature_min_max_dict[i]=np.arange(min_max_df.loc[i,'min'],min_max_df.loc[i,'max']+min_max_df.loc[i,'step'],min_max_df.loc[i,'step'])

In [15]:
psi_bin=pd.DataFrame(columns=['feature','bin','min','=max'])
for i in feature_min_max_dict.keys():
    print(i)
    ix=0
    if i not in numer_feature:
        for j in feature_min_max_dict[i]:
            ix+=1
            psi_bin=psi_bin._append({'feature':i,'bin':j,'min':j,'=max':j},ignore_index=True)
    else:
        for j in feature_min_max_dict[i]:
            ix+=1
            if ix>1 and ix<10:
                psi_bin=psi_bin._append({'feature':i,'bin':ix,'min':j,'=max':feature_min_max_dict[i][ix]},ignore_index=True)
            elif ix==1:
                psi_bin=psi_bin._append({'feature':i,'bin':ix,'min':np.nan,'=max':feature_min_max_dict[i][ix]},ignore_index=True)
            elif ix==10:
                psi_bin=psi_bin._append({'feature':i,'bin':ix,'min':j,'=max':np.nan},ignore_index=True)


SUM_PROMO_last90
SUM_GMV_last90
SUM_PROMO_last30days
SUM_PRDTRX_last30days
SUM_PRDTRX_PROMO_last30days
SUM_GMV_last30days
SUM_GMV_growth
PROMO_ratio_last30days
SUM_PROMO_growth
proba
n_month_active


In [16]:
for i in feature_min_max_dict.keys():
    print(i)
    ix=0
    if i not in numer_feature:
        for j in feature_min_max_dict[i]:
            ix+=1
            train_df.loc[train_df[i]==j,'{}_bin'.format(i)]=j
    else:
        for j in feature_min_max_dict[i]:
            ix+=1
            if ix>1 and ix<10:
                train_df.loc[(train_df[i]>j)&(train_df[i]<=feature_min_max_dict[i][ix]),'{}_bin'.format(i)]=ix
            elif ix==1:
                train_df.loc[train_df[i]<=feature_min_max_dict[i][ix],'{}_bin'.format(i)]=ix
               
            elif ix==10:
                train_df.loc[train_df[i]>j,'{}_bin'.format(i)]=ix

SUM_PROMO_last90
SUM_GMV_last90
SUM_PROMO_last30days
SUM_PRDTRX_last30days
SUM_PRDTRX_PROMO_last30days
SUM_GMV_last30days
SUM_GMV_growth
PROMO_ratio_last30days
SUM_PROMO_growth
proba
n_month_active


In [17]:
train_df

,SUM_PROMO_last90,SUM_GMV_last90,SUM_PROMO_last30days,n_month_active,SUM_PRDTRX_last30days,SUM_PRDTRX_PROMO_last30days,SUM_GMV_last30days,SUM_GMV_growth,PROMO_ratio_last30days,SUM_PROMO_growth,...,SUM_GMV_last90_bin,SUM_PROMO_last30days_bin,SUM_PRDTRX_last30days_bin,SUM_PRDTRX_PROMO_last30days_bin,SUM_GMV_last30days_bin,SUM_GMV_growth_bin,PROMO_ratio_last30days_bin,SUM_PROMO_growth_bin,proba_bin,n_month_active_bin
0,0.000000,0.000000,0.000000,1,0,0,0.000000,0.000000,0.00,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,10.0,1.0
1,0.000000,0.000000,0.000000,1,0,0,0.000000,0.000000,0.00,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,10.0,1.0
2,0.000000,0.000000,0.000000,1,0,0,0.000000,0.000000,0.00,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,10.0,1.0
3,11673.762714,64854.237286,4820.338984,3,2,2,26779.661016,0.000000,0.18,0.000000,...,2.0,1.0,1.0,1.0,1.0,1.0,10.0,1.0,3.0,3.0
4,4735.464408,26308.135592,2012.888136,2,2,2,11182.711864,0.811538,0.18,0.811538,...,1.0,1.0,1.0,1.0,1.0,1.0,10.0,1.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2571,33447.966102,185822.033898,17832.966102,0,15,15,99072.033898,10.030459,0.18,10.030459,...,4.0,3.0,4.0,4.0,3.0,2.0,10.0,2.0,1.0,0.0
2572,18864.000002,104799.999998,10394.847458,0,15,15,57749.152542,1.733944,0.18,1.733944,...,2.0,2.0,4.0,4.0,2.0,1.0,10.0,1.0,1.0,0.0
2573,19297.220341,107206.779659,10248.406780,3,15,15,56935.593220,1.558794,0.18,1.558794,...,2.0,2.0,4.0,4.0,2.0,1.0,10.0,1.0,1.0,3.0
2574,32364.915254,179805.084746,18427.881356,3,16,16,102377.118644,10.365079,0.18,10.365079,...,4.0,3.0,4.0,4.0,3.0,2.0,10.0,2.0,1.0,3.0


In [18]:
psi_bin_percent=pd.DataFrame(columns=['feature','bin','percent_to_total'])
for i in feature_min_max_dict.keys():
    df_new = train_df['{}_bin'.format(i)].value_counts(normalize=True).reset_index().rename(columns={'{}_bin'.format(i):'bin','proportion':'percent_to_total'})
    df_new['feature'] = i
    psi_bin_percent = psi_bin_percent._append(df_new,ignore_index=True)

In [19]:
psi_bin_percent

,feature,bin,percent_to_total
0,SUM_PROMO_last90,1.0,0.641304
1,SUM_PROMO_last90,2.0,0.199534
2,SUM_PROMO_last90,3.0,0.078804
3,SUM_PROMO_last90,4.0,0.035326
4,SUM_PROMO_last90,5.0,0.022904
...,...,...,...
95,proba,9.0,0.011646
96,n_month_active,3.0,0.371118
97,n_month_active,0.0,0.291149
98,n_month_active,1.0,0.222050


In [20]:
psi_bin.merge(psi_bin_percent,on=['feature','bin'],how='left').rename(columns={'=max':'eq_max'})

,feature,bin,min,eq_max,percent_to_total
0,SUM_PROMO_last90,1,NaN,11846.059,0.641304
1,SUM_PROMO_last90,2,11846.059000,23692.118,0.199534
2,SUM_PROMO_last90,3,23692.118000,35538.177,0.078804
3,SUM_PROMO_last90,4,35538.177000,47384.236,0.035326
4,SUM_PROMO_last90,5,47384.236000,59230.295,0.022904
...,...,...,...,...,...
99,proba,10,0.570567,NaN,0.066770
100,n_month_active,0,0.000000,0.000,0.291149
101,n_month_active,1,1.000000,1.000,0.222050
102,n_month_active,2,2.000000,2.000,0.115683


In [21]:
psi_bin.merge(psi_bin_percent,on=['feature','bin'],how='left').rename(columns={'=max':'eq_max'}).to_csv('psi_train_reference.csv')

In [22]:
psi_bin=psi_bin.merge(psi_bin_percent,on=['feature','bin'],how='left')

In [23]:
psi_bin.rename(columns={'=max':'eq_max'},inplace=True)

In [24]:
import pytz
jakarta_tz = pytz.timezone("Asia/Jakarta")
jakarta_time = datetime.now(jakarta_tz)
jakarta_time.strftime("%Y-%m-%d")

'2024-04-08'

In [25]:
# jakarta_time = pd.to_datetime('2022-06-01')

In [26]:
churn_score_result_table = 'churn_pred_score'

In [27]:
project_id_bq+'.'+churn_bq_db+'.'+churn_score_result_table

'testdv.portfolio.churn_pred_score'

In [28]:
date_filter="2022-11-21"
df_psi_count=pd.DataFrame(columns=["feature","bin","count"])
for i in col_to_be_trained+['proba']:
    query="""
    SELECT """
    ix=0
    print(i)
    for j in range(10):
        ix+=1
        if i == "n_month_active":
            query="""
            SELECT 
                COUNT(CASE WHEN n_month_active = 1 THEN 1 END) AS bin_1,
                COUNT(CASE WHEN n_month_active = 2 THEN 1 END) AS bin_2,
                COUNT(CASE WHEN n_month_active = 3 THEN 1 END) AS bin_3,
                COUNT(CASE WHEN n_month_active = 0 THEN 1 END) AS bin_0,
            FROM {}
            WHERE date(week_date) = '{}'
            """.format(project_id_bq+'.'+churn_bq_db+'.'+churn_score_result_table,
                        date_filter)
        else:
            if ix>1 and ix<10:
                query+="""COUNT(CASE WHEN {} > {} AND {} <= {}  THEN 1 END) AS bin_{},
                """.format(
                    psi_bin.loc[(psi_bin["feature"]==i)&(psi_bin["bin"]==ix),"feature"].values[0],
                    psi_bin.loc[(psi_bin["feature"]==i)&(psi_bin["bin"]==ix),"min"].values[0],
                    psi_bin.loc[(psi_bin["feature"]==i)&(psi_bin["bin"]==ix),"feature"].values[0],
                    psi_bin.loc[(psi_bin["feature"]==i)&(psi_bin["bin"]==ix),"eq_max"].values[0],
                    ix
                )
            elif ix==10:
                query+="""COUNT(CASE WHEN {} > {} THEN 1 END) AS bin_{}
                FROM {}
                    WHERE date(week_date) = '{}' 
                """.format(
                    psi_bin.loc[(psi_bin["feature"]==i)&(psi_bin["bin"]==ix),"feature"].values[0],
                    psi_bin.loc[(psi_bin["feature"]==i)&(psi_bin["bin"]==ix),"min"].values[0],
                    ix,
                    project_id_bq+'.'+churn_bq_db+'.'+churn_score_result_table,
                    date_filter
                    
                )
            elif ix==1:
                query+=""" COUNT(CASE WHEN {} <= {} THEN 1 END) AS bin_{},
                """.format(
                    psi_bin.loc[(psi_bin["feature"]==i)&(psi_bin["bin"]==ix),"feature"].values[0],
                    psi_bin.loc[(psi_bin["feature"]==i)&(psi_bin["bin"]==ix),"eq_max"].values[0],
                    ix
                )
    client = bigquery.Client(credentials=credentials, project=project_id_bq)
    job = client.query(query)
    df_bin_count=job.result().to_dataframe()
    df_bin_count=df_bin_count.T.reset_index()
    df_bin_count["index"] = df_bin_count["index"].map(lambda x: x[4:])
    df_bin_count.rename(columns={"index":"bin"},inplace=True)
    df_bin_count["feature"]=i
    df_bin_count.rename(columns={0:"count"},inplace=True)
    df_psi_count=df_psi_count._append(df_bin_count,ignore_index=True)

SUM_PROMO_last90
SUM_GMV_last90
SUM_PROMO_last30days
n_month_active
SUM_PRDTRX_last30days
SUM_PRDTRX_PROMO_last30days
SUM_GMV_last30days
SUM_GMV_growth
PROMO_ratio_last30days
SUM_PROMO_growth
proba


In [29]:
df_psi_count['proportion']=df_psi_count['count'].div(df_psi_count[['feature','count']].groupby(['feature']).sum().reset_index()['count'][0])

In [30]:
df_psi_count[["bin","proportion"]]=df_psi_count[["bin","proportion"]].astype(float)

In [31]:
from datetime import datetime, timedelta
now = datetime.now()
monday = now - timedelta(days = now.weekday())

In [32]:
pd.to_datetime(monday.strftime("%Y-%m-%d"))

Timestamp('2024-04-08 00:00:00')

In [33]:
df_psi_count['week_date']=pd.to_datetime('2022-11-21').strftime("%Y-%m-%d")

In [34]:
# df_psi_count['week_date']=pd.to_datetime(monday.strftime("%Y-%m-%d"))

In [35]:
psi_bin

,feature,bin,min,eq_max,percent_to_total
0,SUM_PROMO_last90,1,NaN,11846.059,0.641304
1,SUM_PROMO_last90,2,11846.059000,23692.118,0.199534
2,SUM_PROMO_last90,3,23692.118000,35538.177,0.078804
3,SUM_PROMO_last90,4,35538.177000,47384.236,0.035326
4,SUM_PROMO_last90,5,47384.236000,59230.295,0.022904
...,...,...,...,...,...
99,proba,10,0.570567,NaN,0.066770
100,n_month_active,0,0.000000,0.000,0.291149
101,n_month_active,1,1.000000,1.000,0.222050
102,n_month_active,2,2.000000,2.000,0.115683


In [36]:
psi_bin_merged = psi_bin.merge(df_psi_count, on=["feature", "bin"])
psi_bin_merged["current_min_train"] = (
    psi_bin_merged["proportion"] - psi_bin_merged["percent_to_total"]
)
psi_bin_merged["ln_current_to_train"] = np.log1p(
    psi_bin_merged["proportion"].div(psi_bin_merged["percent_to_total"])
)
psi_bin_merged["psi"] = psi_bin_merged["ln_current_to_train"] * psi_bin_merged["current_min_train"]
psi_final = (
    psi_bin_merged[["feature", "psi"]]
    .replace(np.inf, 0)
    .groupby("feature")
    .sum()
    .reset_index()
)

In [37]:
psi_final

,feature,psi
0,PROMO_ratio_last30days,0.030956
1,SUM_GMV_growth,0.028414
2,SUM_GMV_last30days,0.163352
3,SUM_GMV_last90,0.171519
4,SUM_PRDTRX_PROMO_last30days,0.122252
5,SUM_PRDTRX_last30days,0.121492
6,SUM_PROMO_growth,0.015475
7,SUM_PROMO_last30days,0.178215
8,SUM_PROMO_last90,0.118427
9,n_month_active,0.070066


In [38]:
psi_final['week_date']=pd.to_datetime('2022-11-21')

In [45]:
psi_final['created_at']=pd.to_datetime('2022-11-21')

In [46]:
job_config = bigquery.LoadJobConfig(
            write_disposition='WRITE_TRUNCATE',
            schema=[
                bigquery.SchemaField("week_date", "DATE"),
                bigquery.SchemaField("psi", "FLOAT"),
            ],
            # time_partitioning=bigquery.TimePartitioning(
            #     type_=bigquery.TimePartitioningType.DAY,
            #     field="week_date",  # field to use for partitioning
            # ),
        )


In [47]:
# store data
churn_psi_result_table ='churn_pred_psi'
client = bigquery.Client(credentials=credentials, project=project_id_bq)
job = client.load_table_from_dataframe(
psi_final
                
                 , churn_bq_db+'.'+churn_psi_result_table, job_config=job_config
)
job.result()  # Wait for the job to complete.

LoadJob<project=testdv, location=asia-southeast2, id=75d86cd2-578e-42d7-9bfc-07ec94b75c86>